In [ ]:
# Import verification
try:
    from src import BigramLM, HangmanEnv, greedy_play, DQNAgent
    print('src imports OK:', BigramLM, HangmanEnv, greedy_play, DQNAgent)
except Exception as e:
    print('Import error:', e)

# Run baseline via evaluate helper
import sys
sys.path.append('..')
from src.evaluate import run_baseline
_ = run_baseline('../data/corpus.txt','../data/test_words.txt',n_games=100,lives=6,seed=42,outdir='../plots', data_dir='../data')

# 02 Baseline Greedy (HMM)
Run the greedy agent over test words and view metrics.

In [ ]:
%pip install -q -r ../requirements.txt
import os, sys
sys.path.append('..')
from src.evaluate import run_baseline
metrics=run_baseline('../data/corpus.txt','../data/test_words.txt',n_games=100,lives=6,seed=42,outdir='../output')
metrics

# Live Demo: Single-Game Playthrough
Use the next cells to play one game step-by-step for the demo.
- Baseline (HMM + Greedy, strict/pure argmax)
- Optional: DQN (strict or hybrid, toggle in the cell)

In [ ]:
# Baseline demo: single game (strict)
import random, time, sys
sys.path.append('..')
from src import BigramLM, greedy_play

oracle = BigramLM(smoothing=1.0); oracle.fit('../data/corpus.txt')
# Choose a word: either random from test set or set a fixed one for deterministic demo
words = [w.strip() for w in open('../data/test_words.txt', encoding='utf-8') if w.strip()]
word = random.choice(words)  # e.g., set: word = 'planet'

res = greedy_play(word, oracle, lives=6, seed=42)
print(f"Word target: {res.word}")
for s in res.steps:
    print(s)
    time.sleep(0.2)  # smooth pacing for demo
print(f"Success: {res.success} | Wrong: {res.wrong_guesses} | Repeated: {res.repeated_guesses}")

In [ ]:
# Optional: DQN demo (toggle HYBRID for nicer behavior)
import os, torch, random, time, sys
sys.path.append('..')
from src.hmm_oracle import HMMOracle
from src.hangman_env import HangmanEnv
from src.dqn_agent import DQNAgent
from src.utils import ALPHABET

HYBRID = False  # set True to allow oracle top-k guidance for demo

oracle = HMMOracle(1.0); oracle.fit('../data/corpus.txt')
words = [w.strip() for w in open('../data/test_words.txt', encoding='utf-8') if w.strip()]
word = random.choice(words)  # or set a fixed word, e.g., 'planet'
max_len = min(20, max(len(word),1))
env = HangmanEnv(word, oracle, lives=6, max_len=max_len)
s = env.reset()
agent = DQNAgent(len(s))
pth = '../data/dqn_agent.pth'
if os.path.exists(pth):
    agent.q.load_state_dict(torch.load(pth, map_location='cpu'))
    agent.tgt.load_state_dict(agent.q.state_dict())
agent.epsilon = 0.0

print('Word target:', word)
wrong = repeated = 0
while True:
    guessed_mask = s[max_len*26:max_len*26+26]
    if HYBRID:
        guessed_set = {ALPHABET[i] for i,m in enumerate(guessed_mask) if m==1.0}
        dist = oracle.letter_distribution(env.pattern, guessed_set)
        allowed = [ALPHABET.index(ch) for ch,_ in sorted(dist.items(), key=lambda x:x[1], reverse=True) if ch not in guessed_set][:5]
        if allowed:
            with torch.no_grad():
                qv = agent.q(torch.tensor(s, dtype=torch.float32).unsqueeze(0)).squeeze(0)
                a = max(allowed, key=lambda idx: qv[idx].item())
        else:
            a = agent.act(s, guessed_mask)
    else:
        a = agent.act(s, guessed_mask)  # strict
    if guessed_mask[a]==1.0: repeated += 1
    sn, r, done, info = env.step(a)
    if not info.get('revealed', False): wrong += 1
    print(f"Word: {env.pattern} | Lives: {env.lives} | Guess: {ALPHABET[a]}")
    time.sleep(0.2)
    s = sn
    if done: break
print('Success:', '_' not in env.pattern, '| Wrong:', wrong, '| Repeated:', repeated)